# Image Processing with nibabel and an External Python Script

This notebook illustrates how you can use an external Python script (in this case, `dog.py`) to perform image processing inside a Jupyter environment. The script loads a NIfTI image from disk, processes it, and returns the result directly as an in-memory array (without writing a new file to disk).

In this example, the grayscale input image is first thresholded using Otsu’s method, and then a Difference of Gaussian ([DoG](https://pubmed.ncbi.nlm.nih.gov/38508496/)) filter is applied to extract a clean binary edge outline.

The goal of this notebook is to demonstrate how to integrate external Python scripts into a Jupyter workflow. Note that similar computations could also be performed using the [niimath WebAssembly (WASM)](https://niivue.github.io/niivue-niimath/) plugin available in NiiVue.

In [ ]:
import ipyniivue
import ipywidgets as widgets
from IPython.display import display
import importlib
import subprocess
import sys

# install required Python packages

def ensure_packages(packages):
    """Provide packages used by dog.py."""
    to_install = []
    for pkg in packages:
        if isinstance(pkg, tuple):
            pip_name, module_name = pkg
        else:
            pip_name = module_name = pkg  # assume pip name == import name
        try:
            importlib.import_module(module_name)
        except Exception:
            to_install.append(pip_name)

    if to_install:
        subprocess.check_call([sys.executable, "-m", "pip", "install", *to_install])


pkgs = [
    ("nibabel", "nibabel"),
    ("numpy", "numpy"),
    ("scipy", "scipy"),
    ("scikit-image", "skimage"),
]

ensure_packages(pkgs)

# Import dog.py after we ensure dependencies are installed
import dog # noqa: E402

# NiiVue settings

nv = ipyniivue.NiiVue(height=512)

fnm_reference = "../images/mni152.nii.gz"
fnm_warped = "../images/T2w_stroke.nii.gz"
volume = ipyniivue.Volume(name="outline.nii", data=dog.dog_ram(fnm_warped), colormap="actc")
nv.load_volumes([
    {"path": fnm_reference},
    {"path": fnm_warped, "colormap": "navia", "opacity": 0},
    volume
])

# user interface

reference_slider = widgets.FloatSlider(
    min=0, max=1, step=0.1, value=1, description="Reference", continuous_update=True, readout=False
)

warped_slider = widgets.FloatSlider(
    min=0, max=1, step=0.1, value=0, description="Warped", continuous_update=True, readout=False
)

outline_slider = widgets.FloatSlider(
    min=0, max=1, step=0.1, value=1, description="Outline", continuous_update=True, readout=False
)

def on_reference_change(change):
    """Modify reference image transparency."""
    nv.volumes[0].opacity = change["new"]

def on_warped_change(change):
    """Modify warped image transparency."""
    nv.volumes[1].opacity = change["new"]

def on_outline_change(change):
    """Modify outline image transparency."""
    nv.volumes[2].opacity = change["new"]

reference_slider.observe(on_reference_change, names="value")
warped_slider.observe(on_warped_change, names="value")
outline_slider.observe(on_outline_change, names="value")

# display

controls = widgets.HBox([reference_slider, warped_slider, outline_slider])
display(widgets.VBox([controls, nv]))